# Claycode SIGGRAPH Evaluation

In [196]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv('results.csv')

# Process the 'type' column to categorize types based on the filename
df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay", "clay"))

# Select relevant columns
df = df[['successful', 'wave_amplitude', 'type']]

# Group by wave amplitude and type, then calculate success rate
grouped = df.groupby(['wave_amplitude', 'type'])
df = grouped['successful'].agg(['sum', 'count']).reset_index()
df['success_rate'] = df['sum'] / df['count']

df = df[df['wave_amplitude'] > 0]

In [197]:
# Define all pairs to display
trace_types = [
   'claycode', 'claycode-fragment','claycodeart', 'qrart','qr','barcode'
]

##################
# Offsets
##################

# Apply small offsets to the success rate for better separation between overlapping claycode types
STEP = 0.005
df['offset_success_rate'] = df['success_rate']
df.loc[df['type'] == 'claycode', 'offset_success_rate'] += 3*STEP
df.loc[df['type'] == 'claycode-fragment', 'offset_success_rate'] += 1*STEP
df.loc[df['type'] == 'claycodeart', 'offset_success_rate'] -= 1*STEP
df.loc[df['type'] == 'barcode', 'offset_success_rate'] -= 3*STEP

##################
# Colors
##################

claycode_color_main = '#156b59'  
claycode_color_variant2 = '#298c73'  
claycode_color_variant1 = '#26a78a' 
qr_color_main = '#d95f02'  # Main shade for qr
qr_color_variant = '#f5861a'  # orange for variant
barcode_color = '#1160b8' 

color_scheme = {
    'claycode': claycode_color_main, 
    'claycodeart': claycode_color_variant1, 
    'claycode-fragment': claycode_color_variant2, 
    'qr': qr_color_main, 
    'qrart': qr_color_variant, 
    'barcode': barcode_color,
    'claycode2': claycode_color_main, 
    'claycodeart2': claycode_color_variant1, 
    'claycode2-fragment': claycode_color_variant2, 
    'qr2': qr_color_main, 
    'qrart2': qr_color_variant, 
    'barcode2': barcode_color
}

##################
# Marker styles
##################

marker_qr = 'square'
marker_qrart = 'diamond'
marker_claycode = 'circle'
marker_claycodeart = 'bowtie'
marker_claycode_fragment = 'diamond-tall'
marker_barcode = 'x'
marker_map = {
    'qr': marker_qr,
    'qrart': marker_qrart,
    'claycode': marker_claycode,
    'claycodeart': marker_claycodeart,
    'barcode': marker_barcode,
    'claycode-fragment': marker_claycode_fragment,
}

MULT = 1.1
SIZE_QR = 8
SIZE_QR_ART = 11
SIZE_CLAYCODE = 11
SIZE_CLAYCODE_ART = 15
SIZE_CLAYCODE_FRAGMENT = 12
SIZE_BARCODE = 9
marker_size_map = {
    'claycode': SIZE_CLAYCODE,
    'claycodeart': SIZE_CLAYCODE_ART,
    'claycode-fragment': SIZE_CLAYCODE_FRAGMENT,
    'qr': SIZE_QR,
    'qrart': SIZE_QR_ART,
    'barcode': SIZE_BARCODE,
}
marker_size_map = { key: value*MULT for key,value in marker_size_map.items() }


# Plot setup
fig = go.Figure()

for trace_type in trace_types:
    trace = df[df['type'] == trace_type]
    fig.add_trace(go.Scatter(
        x=trace['wave_amplitude'],
        y=trace['offset_success_rate'],
        mode='lines+markers',
        name=trace_type,
        line=dict(
            color=color_scheme.get(trace_type, '#000000'),
        ),
        marker=dict(
            symbol=marker_map.get(trace_type, 'circle'),
            size=marker_size_map.get(trace_type, 0)
        )
    ))

FIG_MULT = 0.8
fig.update_layout(
    xaxis_title="Wave Amplitude (ω)",
    height=410*FIG_MULT,
    width=615*FIG_MULT,
    showlegend=False,
    margin=dict(t=10, b=10, l=10, r=10)
)

fig.update_traces(opacity=1, line=dict(width=2.6))
fig.update_xaxes(range=[0.08, 1.02], tickformat='.00', dtick=0.1)
fig.update_yaxes(range=[-.035, 1.05], tickformat='.0%')

# Export to PDF
fig.write_image("deformation.pdf", format="pdf", engine="kaleido")
fig.show()


In [198]:
# y: every kind of string (group by message name)
# x: amplitudine
# y: type of code

df = pd.read_csv('results.csv')

# Extract the string between the last '-' and '.png' in the filename
df['extracted_value'] = df['filename'].str.extract(r'-([^-]+)\.png$')

# Calculate the length of the extracted value and use it for the Y-axis
df['extracted_value_len'] = df['extracted_value'].apply(len)

# Sort by the length of the extracted value
df = df.sort_values('extracted_value_len')

# Group by 'extracted_value_len' and 'wave_amplitude', then count the successful entries
grouped_df = df.groupby(['extracted_value_len', 'wave_amplitude'], as_index=False).agg({
    'successful': 'sum'  # Count of successful rows (True = 1, False = 0)
})

x_max = grouped_df['wave_amplitude'].max()
if x_max != 1:
    x_max = 1

# Create the heatmap with length of extracted value on Y-axis
fig = px.density_heatmap(
    grouped_df,
    x='wave_amplitude',
    y='extracted_value_len',
    z='successful',
    title='Heatmap of Wave Amplitude and Length of Extracted Filename Value (Count of Successful)',
    labels={
        'wave_amplitude': 'Wave Amplitude',
        'extracted_value_len': 'Length of Extracted Value',
        'successful': 'Count of Successful Entries'
    },
    color_continuous_scale='Teal',
)

fig.update_layout(
    xaxis=dict(range=[0, 1]),
)

# Show the plot
fig.show()